# Deploy Red Bull Receipt OCR API (Colab)

⚠️ IMPORTANT: Set Runtime to GPU (T4, L4, or A100).\n\nThis notebook creates a FastAPI server (with optional public URL via ngrok) for scanning receipts using your fine-tuned model.

In [ ]:
# -*- coding: utf-8 -*-

# ===========================
# STEP 1: Install Dependencies
# ===========================
print("="*60)
print("📦 INSTALLING DEPENDENCIES")
print("="*60)

!pip install -q transformers torch torchvision pillow
!pip install -q fastapi uvicorn pyngrok python-multipart
!pip install -q timm einops

print("✓ Dependencies installed")


In [ ]:
# ===========================
# STEP 2: Mount Drive and Load Model
# ===========================
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import torch
from pathlib import Path
from PIL import Image
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

print("\n" + "="*60)
print("🤖 LOADING MODEL")
print("="*60)

MODEL_PATH = "/content/drive/MyDrive/Scan Bill/trained_model"

if not os.path.exists(MODEL_PATH):
    print(f"❌ Model not found at: {MODEL_PATH}")
    print("\nPlease check your model location or update MODEL_PATH")
    raise FileNotFoundError("Model not found")

print(f"Loading model from: {MODEL_PATH}")

# Load model
model = AutoModel.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map='auto'
).eval()

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH, 
    trust_remote_code=True, 
    use_fast=False
)

print("✓ Model loaded successfully")
print(f"  Device: {next(model.parameters()).device}")
if torch.cuda.is_available():
    print(f"  Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
else:
    print("  Memory: CUDA not available")


In [ ]:
# ===========================
# STEP 3: Image Processing Functions
# ===========================
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) 
        for j in range(1, n + 1) if i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    target_width = int(image_size * target_aspect_ratio[0])
    target_height = int(image_size * target_aspect_ratio[1])
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    
    return processed_images

def load_image(image, input_size=448, max_num=12):
    """Process PIL Image for model inference"""
    if isinstance(image, str):
        image = Image.open(image).convert('RGB')
    
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(img) for img in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

def extract_receipt_info(image):
    """Extract receipt information from image"""
    try:
        # Process image
        pixel_values = load_image(image, max_num=12).to(torch.bfloat16)
        if torch.cuda.is_available():
            pixel_values = pixel_values.cuda()
        
        # Generate response
        question = "<image>\nTrích xuất thông tin hóa đơn trong ảnh dưới dạng JSON."
        
        with torch.no_grad():
            response = model.chat(
                tokenizer, 
                pixel_values, 
                question, 
                generation_config=dict(
                    max_new_tokens=1024, 
                    do_sample=False,
                    num_beams=3,
                    repetition_penalty=2.0
                )
            )
        
        # Try to parse as JSON
        try:
            # Clean response if it has markdown code blocks
            if isinstance(response, str) and response.startswith('```'):
                response = response.split('```')[1]
                if response.startswith('json'):
                    response = response[4:]
            
            result = json.loads(response) if isinstance(response, str) else response
            return {"success": True, "data": result, "raw_response": response}
        except json.JSONDecodeError:
            # Return raw response if not valid JSON
            return {"success": True, "data": None, "raw_response": response}
            
    except Exception as e:
        return {"success": False, "error": str(e)}

print("\n✓ Image processing functions ready")


In [ ]:
# ===========================
# STEP 4: Create FastAPI Server
# ===========================
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import io

app = FastAPI(
    title="Red Bull Receipt OCR API",
    description="Extract receipt information from images using fine-tuned Vintern model",
    version="1.0.0"
)

# Enable CORS for all origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    """API root endpoint"""
    return {
        "message": "Red Bull Receipt OCR API",
        "version": "1.0.0",
        "endpoints": {
            "POST /scan": "Upload image to extract receipt information",
            "POST /scan_base64": "Send base64 encoded image",
            "GET /health": "Check API health status"
        },
        "model": "Vintern-1B-v3_5 (Fine-tuned on Red Bull receipts)"
    }

@app.get("/health")
async def health():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "model_loaded": model is not None,
        "gpu_available": torch.cuda.is_available(),
        "gpu_memory_used_gb": (torch.cuda.memory_allocated() / 1024**3) if torch.cuda.is_available() else 0
    }

@app.post("/scan")
async def scan_receipt(file: UploadFile = File(...)):
    """
    Scan receipt image and extract information
    
    Args:
        file: Image file (jpg, png, jpeg, bmp)
    
    Returns:
        JSON with extracted receipt information
    """
    # Validate file type
    if not file.content_type.startswith('image/'):
        raise HTTPException(status_code=400, detail="File must be an image")
    
    try:
        # Read image
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))
        
        # Extract information
        result = extract_receipt_info(image)
        
        if result["success"]:
            return JSONResponse(content={
                "success": True,
                "filename": file.filename,
                "extracted_data": result.get("data"),
                "raw_response": result.get("raw_response")
            })
        else:
            raise HTTPException(status_code=500, detail=result["error"])
            
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing image: {str(e)}")

@app.post("/scan_base64")
async def scan_receipt_base64(data: dict):
    """
    Scan receipt from base64 encoded image
    
    Args:
        data: {"image": "base64_encoded_string"}
    
    Returns:
        JSON with extracted receipt information
    """
    try:
        import base64
        
        if "image" not in data:
            raise HTTPException(status_code=400, detail="Missing 'image' field in request body")
        
        # Decode base64
        image_data = base64.b64decode(data["image"])
        image = Image.open(io.BytesIO(image_data))
        
        # Extract information
        result = extract_receipt_info(image)
        
        if result["success"]:
            return JSONResponse(content={
                "success": True,
                "extracted_data": result.get("data"),
                "raw_response": result.get("raw_response")
            })
        else:
            raise HTTPException(status_code=500, detail=result["error"])
            
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing image: {str(e)}")

print("\n✓ FastAPI server created")


In [ ]:
# ===========================
# STEP 5: Setup ngrok for Public URL
# ===========================
from pyngrok import ngrok, conf
import getpass

print("\n" + "="*60)
print("🌐 SETTING UP PUBLIC URL")
print("="*60)

# Get ngrok auth token
print("\n📝 To get a public URL, you need an ngrok auth token:")
print("   1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
print("   2. Sign up (free)")
print("   3. Copy your authtoken")
print()

ngrok_token = getpass.getpass("Enter your ngrok authtoken (or press Enter to skip): ")

if ngrok_token:
    conf.get_default().auth_token = ngrok_token
    print("✓ ngrok token configured")
else:
    print("⚠️ Running without ngrok - only local access available")


In [ ]:
# ===========================
# STEP 6: Start Server
# ===========================
import nest_asyncio
import uvicorn
from threading import Thread

nest_asyncio.apply()

print("\n" + "="*60)
print("🚀 STARTING API SERVER")
print("="*60)

# Start ngrok tunnel if token provided
public_url = None
if ngrok_token:
    try:
        # Kill existing tunnels
        ngrok.kill()
        
        # Create new tunnel
        public_url = ngrok.connect(8000)
        print(f"\n✅ PUBLIC URL: {public_url}")
        print(f"   Share this URL with anyone to access your API!")
    except Exception as e:
        print(f"⚠️ Could not create ngrok tunnel: {e}")
        print("   Server will run locally only")

print(f"\n📍 Local URL: http://localhost:8000")
print(f"\n" + "="*60)
print("API ENDPOINTS:")
print("="*60)
if public_url:
    print(f"POST {public_url}/scan - Upload image file")
    print(f"POST {public_url}/scan_base64 - Send base64 image")
    print(f"GET  {public_url}/health - Health check")
    print(f"GET  {public_url}/docs - Interactive API documentation")
else:
    print("POST http://localhost:8000/scan - Upload image file")
    print("POST http://localhost:8000/scan_base64 - Send base64 image")
    print("GET  http://localhost:8000/health - Health check")
    print("GET  http://localhost:8000/docs - Interactive API documentation")

print("\n" + "="*60)
print("📖 USAGE EXAMPLES")
print("="*60)

print("\n1️⃣ Using cURL:")
if public_url:
    print(f'''
curl -X POST "{public_url}/scan" \n  -F "file=@receipt.jpg"
''')
else:
    print('''
curl -X POST "http://localhost:8000/scan" \
  -F "file=@receipt.jpg"
''')

print("\n2️⃣ Using Python:")
print(f'''
import requests

url = "{public_url or 'http://localhost:8000'}/scan"
files = {{'file': open('receipt.jpg', 'rb')}}
response = requests.post(url, files=files)
print(response.json())
''')

print("\n3️⃣ Using JavaScript:")
print(f'''
const formData = new FormData();
formData.append('file', fileInput.files[0]);

fetch('{public_url or 'http://localhost:8000'}/scan', {{
  method: 'POST',
  body: formData
}})
.then(response => response.json())
.then(data => console.log(data));
''')

print("\n" + "="*60)
print("⚡ SERVER RUNNING - Press Stop/Interrupt to stop")
print("="*60 + "\n")

# Start server (blocking)
config = uvicorn.Config(
    app=app,
    host="0.0.0.0",
    port=8000,
    log_level="info"
)
server = uvicorn.Server(config)
server.run()

# ===========================
# NOTE: Server runs until interrupted
# ===========================
